## 작업형 제1유형 : 데이터 처리 영역
mtcars 데이터셋(mtcars.csv)의 qsec 컬럼을 최소최대 척도(Min-Max Scale)로 변환한 후 0.5보다 
큰 값을 가지는 레코드 수를 구하시오

In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

df = pd.read_csv('data/mtcars.csv',index_col=0)

scaler = MinMaxScaler()
scaled = scaler.fit_transform(df)
scaled_df = pd.DataFrame(data=scaled, columns=df.columns)
print(len(scaled_df[scaled_df.qsec > 0.5]))

9


## 작업형 제2유형 : 모형 구축 및 평가 영역
 아래는 백화점 고객의 1년 간 구매 데이터이다.
 
(가) 제공 데이터 목록  
① y_train.csv : 고객의 성별 데이터 (학습용), CSV 형식의 파일  
② X_train.csv, X_test.csv : 고객의 상품구매 속성 (학습용 및 평가용), CSV 형식의 파일  
(나) 데이터 형식 및 내용  
① y_train.csv (3,500명 데이터)  
② X_train.csv (3,500명 데이터), X_test.csv (2,482명 데이터)  

고객 3,500명에 대한 학습용 데이터(y_train.csv, X_train.csv)를 이용하여 성별예측 모형을 만든 
후, 이를 평가용 데이터(X_test.csv)에 적용하여 얻은 2,482명 고객의 성별 예측값(남자일 확률)을 
다음과 같은 형식의 CSV 파일로 생성하시오.(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 
채점)  

<제출형식>
 custid,gender  
 3500,0.267  
 3501,0.578  
 3502,0.885  
 ․ ․ ․ 
 
 <유의사항>  
성능이 우수한 예측모형을 구축하기 위해서는 적절한 데이터 전처리, Feature Engineering, 분류 
알고리즘 사용, 초매개변수 최적화, 모형 앙상블 등이 수반되어야 한다

In [2]:
# 출력을 원하실 경우 print() 활용
# 예) print(df.head())

# 답안 제출 예시
# 수험번호.csv 생성
# DataFrame.to_csv("0000.csv", index=False)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

X_data = pd.read_csv('./X_train.csv',encoding='CP949')
y_data = pd.read_csv('./y_train.csv',encoding='CP949')

# 데이터 전처리 및 Feature Engineering

# 불필요한 컬럼 및 독립변수간 상관성이 높은 컬럼 삭제
X_data = X_data.drop(['cust_id','최대구매액'],axis=1)

# 환불금액으로 부터 고객의 환불 경향을 반영하는 파생변수 '환불' 도출
X_data['환불금액'].fillna(0, inplace=True)
X_data['환불'] = X_data.환불금액.apply(lambda x:1 if x > 0 else 0)
X_data.drop('환불금액',axis=1,inplace=True)

# 문자형 컬럼 인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
features = ['주구매상품','주구매지점']
for feature in features:
    X_data[feature] = le.fit_transform(X_data[feature])

# 내점일수, 내점당 구매건수, 구매주기 컬럼 범주화
for i in range(len(X_data)):
  if X_data.내점일수[i] <= 2:
    X_data.내점일수[i] = 1
  elif X_data.내점일수[i] <= 8:
    X_data.내점일수[i] = 2
  elif X_data.내점일수[i] <= 25:
    X_data.내점일수[i] = 3
  else :
    X_data.내점일수[i] = 4

for i in range(len(X_data)):
  if X_data.내점당구매건수[i] <= 1.7:
    X_data.내점당구매건수[i] = 1
  elif X_data.내점당구매건수[i] <= 2.3:
    X_data.내점당구매건수[i] = 2
  elif X_data.내점당구매건수[i] <= 3.4:
    X_data.내점당구매건수[i] = 3  
  else :
    X_data.내점당구매건수[i] = 4

for i in range(len(X_data)):
  if X_data.구매주기[i] <= 4:
    X_data.구매주기[i] = 1
  elif X_data.구매주기[i] <= 13:
    X_data.구매주기[i] = 2
  elif X_data.구매주기[i] <= 28:
    X_data.구매주기[i] = 3  
  else :
    X_data.구매주기[i] = 4
    
# 스케일링
from sklearn.preprocessing import MinMaxScaler    
features = ['총구매액']
scaler = MinMaxScaler()
for feature in features:
    X_data[feature] = scaler.fit_transform(X_data[[feature]])

y_data=y_data['gender']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_data,y_data,test_size=0.2, random_state=10)

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score 

# 분류 모델 적용
dt_model = DecisionTreeClassifier()
lr_model = LogisticRegression()
dt_model.fit(X_train,y_train)
lr_model.fit(X_train,y_train)
dt_pred_proba = dt_model.predict_proba(X_test)[:,1]
lr_pred_proba = lr_model.predict_proba(X_test)[:,1]
roc_dt = roc_auc_score(y_test,dt_pred_proba)
roc_lr = roc_auc_score(y_test,lr_pred_proba)
print('dt auc :', roc_dt)
print('lr auc :', roc_lr)

# 앙상블 모형인 랜덤 포레스트 적용(초매개 변수 최적화)
# rf_model = RandomForestClassifier(random_state=0)
rf_model = RandomForestClassifier(random_state=0, n_estimators = 350,max_depth=6,min_samples_leaf=4)
rf_model.fit(X_train,y_train)
rf_pred_proba = rf_model.predict_proba(X_test)[:,1]
roc_rf = roc_auc_score(y_test, rf_pred_proba)
print('rf auc :', roc_rf)

dt auc : 0.5420344682639765
lr auc : 0.651229722658294
rf auc : 0.6787751670655653


In [4]:
#3500개 데이터중 남성 데이터의 수 확인(약 3분의 1에 해당함)
sum(y_data ==1)

1316

In [2]:
# GridSearchCV를 사용 - RF 알고리즘, 교차검증, 하이퍼파라미터 튜닝
from sklearn.model_selection import GridSearchCV
params = {'n_estimators':[300,350,400],
         'max_depth':[10,12,15,20],
         'min_samples_leaf':[6,8,10],
         'min_samples_split':[6,8,10]}
rf_model = RandomForestClassifier(random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(rf_model, param_grid=params, cv=3,n_jobs=-1)
grid_cv.fit(X_train,y_train)

print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)

best_gr_clf = grid_cv.best_estimator_
pred_proba = best_gr_clf.predict_proba(X_test)[:,1]
auc = roc_auc_score(y_test,pred_proba)
print('auc:{0:.4f}'.format(auc))

최적 하이퍼 파라미터:
 {'max_depth': 20, 'min_samples_leaf': 8, 'min_samples_split': 6, 'n_estimators': 400}
auc:0.6796


In [19]:
# XGBoost 알고리즘

from xgboost import XGBClassifier
evals = [(X_test, y_test)]
xgb_wrapper = XGBClassifier(n_estimators=400, learning_rate=0.1)
xgb_wrapper.fit(X_train,y_train, early_stopping_rounds=100,eval_set=evals, 
                eval_metric='logloss',verbose=True)
w_preds = xgb_wrapper.predict(X_test)
w_preds_proba = xgb_wrapper.predict_proba(X_test)[:,1]
roc_xgb = roc_auc_score(y_test, w_preds_proba)
print(roc_xgb)

[0]	validation_0-logloss:0.681267
Will train until validation_0-logloss hasn't improved in 100 rounds.
[1]	validation_0-logloss:0.672536
[2]	validation_0-logloss:0.665129
[3]	validation_0-logloss:0.659021
[4]	validation_0-logloss:0.653923
[5]	validation_0-logloss:0.649877
[6]	validation_0-logloss:0.64651
[7]	validation_0-logloss:0.643619
[8]	validation_0-logloss:0.641723
[9]	validation_0-logloss:0.639418
[10]	validation_0-logloss:0.637496
[11]	validation_0-logloss:0.636924
[12]	validation_0-logloss:0.635052
[13]	validation_0-logloss:0.634487
[14]	validation_0-logloss:0.633904
[15]	validation_0-logloss:0.633357
[16]	validation_0-logloss:0.632549
[17]	validation_0-logloss:0.631742
[18]	validation_0-logloss:0.630309
[19]	validation_0-logloss:0.629509
[20]	validation_0-logloss:0.629358
[21]	validation_0-logloss:0.629478
[22]	validation_0-logloss:0.629301
[23]	validation_0-logloss:0.628606
[24]	validation_0-logloss:0.628891
[25]	validation_0-logloss:0.628741
[26]	validation_0-logloss:0.6287

In [20]:
# GridSearchCV를 사용 - XGBoost 알고리즘, 교차검증, 하이퍼파라미터 튜닝
params = {'max_depth':[2,3,5,7],
            'min_child_weight':[1,3],
            'colsample_bytree':[0.2, 0.5,0.75]
            }

grid_cv = GridSearchCV(xgb_wrapper, param_grid=params, cv=5)
grid_cv.fit(X_train,y_train,early_stopping_rounds=30,eval_metric='auc',
eval_set=[(X_train,y_train),(X_test,y_test)])

print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)

best_gr_clf = grid_cv.best_estimator_
pred_proba = best_gr_clf.predict_proba(X_test)[:,1]
auc = roc_auc_score(y_test,pred_proba)
print('auc:{0:.4f}'.format(auc))

[0]	validation_0-auc:0.626543	validation_1-auc:0.626833
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 30 rounds.
[1]	validation_0-auc:0.620821	validation_1-auc:0.615462
[2]	validation_0-auc:0.635561	validation_1-auc:0.629273
[3]	validation_0-auc:0.637425	validation_1-auc:0.631023
[4]	validation_0-auc:0.635779	validation_1-auc:0.630358
[5]	validation_0-auc:0.630645	validation_1-auc:0.625451
[6]	validation_0-auc:0.629488	validation_1-auc:0.624353
[7]	validation_0-auc:0.629122	validation_1-auc:0.624173
[8]	validation_0-auc:0.639073	validation_1-auc:0.633305
[9]	validation_0-auc:0.641545	validation_1-auc:0.632366
[10]	validation_0-auc:0.644717	validation_1-auc:0.636964
[11]	validation_0-auc:0.643369	validation_1-auc:0.635257
[12]	validation_0-auc:0.643371	validation_1-auc:0.634682
[13]	validation_0-auc:0.642535	validation_1-auc:0.635119
[14]	validation_0-auc:0.642278	validation_1-auc:0.63403

[141]	validation_0-auc:0.704149	validation_1-auc:0.678501
[142]	validation_0-auc:0.704288	validation_1-auc:0.678158
[143]	validation_0-auc:0.704244	validation_1-auc:0.678089
[144]	validation_0-auc:0.704236	validation_1-auc:0.678072
[145]	validation_0-auc:0.70429	validation_1-auc:0.678346
[146]	validation_0-auc:0.704371	validation_1-auc:0.678192
[147]	validation_0-auc:0.704212	validation_1-auc:0.678097
[148]	validation_0-auc:0.704573	validation_1-auc:0.678029
[149]	validation_0-auc:0.70444	validation_1-auc:0.677892
[150]	validation_0-auc:0.704888	validation_1-auc:0.678115
[151]	validation_0-auc:0.704892	validation_1-auc:0.678046
[152]	validation_0-auc:0.704887	validation_1-auc:0.677789
[153]	validation_0-auc:0.705437	validation_1-auc:0.678501
[154]	validation_0-auc:0.705402	validation_1-auc:0.678535
[155]	validation_0-auc:0.705395	validation_1-auc:0.678621
[156]	validation_0-auc:0.705377	validation_1-auc:0.678398
[157]	validation_0-auc:0.705368	validation_1-auc:0.678183
[158]	validation

[95]	validation_0-auc:0.69649	validation_1-auc:0.672766
[96]	validation_0-auc:0.696534	validation_1-auc:0.67262
[97]	validation_0-auc:0.696758	validation_1-auc:0.672586
[98]	validation_0-auc:0.69705	validation_1-auc:0.672431
[99]	validation_0-auc:0.69698	validation_1-auc:0.672697
[100]	validation_0-auc:0.696896	validation_1-auc:0.672723
[101]	validation_0-auc:0.696947	validation_1-auc:0.672809
[102]	validation_0-auc:0.69692	validation_1-auc:0.672749
[103]	validation_0-auc:0.697452	validation_1-auc:0.672989
[104]	validation_0-auc:0.697411	validation_1-auc:0.672672
[105]	validation_0-auc:0.697623	validation_1-auc:0.673272
[106]	validation_0-auc:0.697937	validation_1-auc:0.673469
[107]	validation_0-auc:0.69811	validation_1-auc:0.67395
[108]	validation_0-auc:0.699017	validation_1-auc:0.674036
[109]	validation_0-auc:0.699767	validation_1-auc:0.674662
[110]	validation_0-auc:0.699776	validation_1-auc:0.674224
[111]	validation_0-auc:0.699782	validation_1-auc:0.674036
[112]	validation_0-auc:0.7

[49]	validation_0-auc:0.685047	validation_1-auc:0.66692
[50]	validation_0-auc:0.68613	validation_1-auc:0.668146
[51]	validation_0-auc:0.68695	validation_1-auc:0.668961
[52]	validation_0-auc:0.687026	validation_1-auc:0.66891
[53]	validation_0-auc:0.686696	validation_1-auc:0.6687
[54]	validation_0-auc:0.687089	validation_1-auc:0.668974
[55]	validation_0-auc:0.687934	validation_1-auc:0.669275
[56]	validation_0-auc:0.687943	validation_1-auc:0.66912
[57]	validation_0-auc:0.688783	validation_1-auc:0.669069
[58]	validation_0-auc:0.689722	validation_1-auc:0.669926
[59]	validation_0-auc:0.689477	validation_1-auc:0.669961
[60]	validation_0-auc:0.690084	validation_1-auc:0.669944
[61]	validation_0-auc:0.690073	validation_1-auc:0.669592
[62]	validation_0-auc:0.690148	validation_1-auc:0.669575
[63]	validation_0-auc:0.690158	validation_1-auc:0.669232
[64]	validation_0-auc:0.690208	validation_1-auc:0.66918
[65]	validation_0-auc:0.690376	validation_1-auc:0.66924
[66]	validation_0-auc:0.690334	validatio

[48]	validation_0-auc:0.682401	validation_1-auc:0.66867
[49]	validation_0-auc:0.682448	validation_1-auc:0.668713
[50]	validation_0-auc:0.683046	validation_1-auc:0.668695
[51]	validation_0-auc:0.683359	validation_1-auc:0.667898
[52]	validation_0-auc:0.684119	validation_1-auc:0.668258
[53]	validation_0-auc:0.683893	validation_1-auc:0.668112
[54]	validation_0-auc:0.684577	validation_1-auc:0.667713
[55]	validation_0-auc:0.685095	validation_1-auc:0.666812
[56]	validation_0-auc:0.685167	validation_1-auc:0.666787
[57]	validation_0-auc:0.685379	validation_1-auc:0.666701
[58]	validation_0-auc:0.685704	validation_1-auc:0.666384
[59]	validation_0-auc:0.685533	validation_1-auc:0.666692
[60]	validation_0-auc:0.685928	validation_1-auc:0.666615
[61]	validation_0-auc:0.685935	validation_1-auc:0.666281
[62]	validation_0-auc:0.685999	validation_1-auc:0.666315
[63]	validation_0-auc:0.686046	validation_1-auc:0.666066
[64]	validation_0-auc:0.686032	validation_1-auc:0.665886
[65]	validation_0-auc:0.686408	v

...

[37]	validation_0-auc:0.873946	validation_1-auc:0.65165
[38]	validation_0-auc:0.87402	validation_1-auc:0.65153
[39]	validation_0-auc:0.874036	validation_1-auc:0.651573
[40]	validation_0-auc:0.874204	validation_1-auc:0.651401
[41]	validation_0-auc:0.874504	validation_1-auc:0.651238
Stopping. Best iteration:
[11]	validation_0-auc:0.828645	validation_1-auc:0.658221

[0]	validation_0-auc:0.727292	validation_1-auc:0.603452
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 30 rounds.
[1]	validation_0-auc:0.748527	validation_1-auc:0.601303
[2]	validation_0-auc:0.768388	validation_1-auc:0.6216
[3]	validation_0-auc:0.772024	validation_1-auc:0.626584
[4]	validation_0-auc:0.774332	validation_1-auc:0.615929
[5]	validation_0-auc:0.781975	validation_1-auc:0.615651
[6]	validation_0-auc:0.789897	validation_1-auc:0.619777
[7]	validation_0-auc:0.799863	validation_1-auc:0.627201
[8]	validation_0-auc:0.8085	val

[36]	validation_0-auc:0.867933	validation_1-auc:0.635724
[37]	validation_0-auc:0.868794	validation_1-auc:0.637281
[38]	validation_0-auc:0.869704	validation_1-auc:0.6365
[39]	validation_0-auc:0.869627	validation_1-auc:0.636295
[40]	validation_0-auc:0.869841	validation_1-auc:0.635703
[41]	validation_0-auc:0.871964	validation_1-auc:0.633713
[42]	validation_0-auc:0.87372	validation_1-auc:0.633266
[43]	validation_0-auc:0.874666	validation_1-auc:0.634793
[44]	validation_0-auc:0.875108	validation_1-auc:0.634991
[45]	validation_0-auc:0.875353	validation_1-auc:0.634425
[46]	validation_0-auc:0.875525	validation_1-auc:0.633953
[47]	validation_0-auc:0.87747	validation_1-auc:0.63451
[48]	validation_0-auc:0.877815	validation_1-auc:0.633944
[49]	validation_0-auc:0.880001	validation_1-auc:0.634107
[50]	validation_0-auc:0.880197	validation_1-auc:0.633567
[51]	validation_0-auc:0.881831	validation_1-auc:0.633725
[52]	validation_0-auc:0.882418	validation_1-auc:0.634
[53]	validation_0-auc:0.883002	validati

Will train until validation_1-auc hasn't improved in 30 rounds.
[1]	validation_0-auc:0.742395	validation_1-auc:0.62416
[2]	validation_0-auc:0.759759	validation_1-auc:0.647322
[3]	validation_0-auc:0.766065	validation_1-auc:0.656381
[4]	validation_0-auc:0.775178	validation_1-auc:0.649098
[5]	validation_0-auc:0.775153	validation_1-auc:0.645795
[6]	validation_0-auc:0.781214	validation_1-auc:0.648502
[7]	validation_0-auc:0.787897	validation_1-auc:0.654284
[8]	validation_0-auc:0.794463	validation_1-auc:0.654245
[9]	validation_0-auc:0.797191	validation_1-auc:0.656008
[10]	validation_0-auc:0.801815	validation_1-auc:0.651341
[11]	validation_0-auc:0.805523	validation_1-auc:0.649098
[12]	validation_0-auc:0.806514	validation_1-auc:0.651856
[13]	validation_0-auc:0.811341	validation_1-auc:0.653319
[14]	validation_0-auc:0.813366	validation_1-auc:0.653765
[15]	validation_0-auc:0.815597	validation_1-auc:0.653439
[16]	validation_0-auc:0.818138	validation_1-auc:0.655317
[17]	validation_0-auc:0.819915	val

[25]	validation_0-auc:0.832207	validation_1-auc:0.652371
[26]	validation_0-auc:0.83555	validation_1-auc:0.652062
[27]	validation_0-auc:0.837877	validation_1-auc:0.652963
[28]	validation_0-auc:0.838579	validation_1-auc:0.652705
[29]	validation_0-auc:0.839057	validation_1-auc:0.653503
[30]	validation_0-auc:0.840755	validation_1-auc:0.653289
[31]	validation_0-auc:0.842826	validation_1-auc:0.652448
[32]	validation_0-auc:0.844153	validation_1-auc:0.654001
[33]	validation_0-auc:0.845598	validation_1-auc:0.655167
[34]	validation_0-auc:0.846958	validation_1-auc:0.655116
[35]	validation_0-auc:0.847486	validation_1-auc:0.654378
[36]	validation_0-auc:0.850934	validation_1-auc:0.654395
[37]	validation_0-auc:0.852296	validation_1-auc:0.655493
[38]	validation_0-auc:0.852985	validation_1-auc:0.656034
[39]	validation_0-auc:0.855278	validation_1-auc:0.656994
[40]	validation_0-auc:0.855995	validation_1-auc:0.656411
[41]	validation_0-auc:0.856543	validation_1-auc:0.655905
[42]	validation_0-auc:0.857477	v

[96]	validation_0-auc:0.720796	validation_1-auc:0.679817
[97]	validation_0-auc:0.720885	validation_1-auc:0.679809
[98]	validation_0-auc:0.721147	validation_1-auc:0.679689
[99]	validation_0-auc:0.721179	validation_1-auc:0.679903
[100]	validation_0-auc:0.721302	validation_1-auc:0.680195
[101]	validation_0-auc:0.721356	validation_1-auc:0.68004
[102]	validation_0-auc:0.72164	validation_1-auc:0.680298
[103]	validation_0-auc:0.721817	validation_1-auc:0.680083
[104]	validation_0-auc:0.721969	validation_1-auc:0.679414
[105]	validation_0-auc:0.722436	validation_1-auc:0.679165
[106]	validation_0-auc:0.722502	validation_1-auc:0.679054
[107]	validation_0-auc:0.7227	validation_1-auc:0.678642
[108]	validation_0-auc:0.722702	validation_1-auc:0.678419
[109]	validation_0-auc:0.722915	validation_1-auc:0.678462
[110]	validation_0-auc:0.723165	validation_1-auc:0.679204
[111]	validation_0-auc:0.723446	validation_1-auc:0.679346
[112]	validation_0-auc:0.723745	validation_1-auc:0.679654
[113]	validation_0-auc

In [9]:
# 고객의 성별 예측값(남자일 확률)

X_df = pd.read_csv('./X_test.csv',encoding='CP949')

# 불필요한 컬럼 및 독립변수간 상관성이 높은 컬럼 삭제
X_test = X_df.drop(['cust_id','최대구매액'],axis=1)

# 환불금액으로 부터 고객의 환불 경향을 반영하는 파생변수 '환불' 도출
X_test['환불금액'].fillna(0, inplace=True)
X_test['환불'] = X_test.환불금액.apply(lambda x:1 if x > 0 else 0)
X_test.drop('환불금액',axis=1,inplace=True)

# 문자형 컬럼 인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
features = ['주구매상품','주구매지점']
for feature in features:
    X_test[feature] = le.fit_transform(X_test[feature])

# 내점일수, 내점당 구매건수, 구매주기 컬럼 범주화
for i in range(len(X_test)):
  if X_test.내점일수[i] <= 2:
    X_test.내점일수[i] = 1
  elif X_test.내점일수[i] <= 8:
    X_test.내점일수[i] = 2
  elif X_test.내점일수[i] <= 25:
    X_test.내점일수[i] = 3
  else :
    X_test.내점일수[i] = 4

for i in range(len(X_test)):
  if X_test.내점당구매건수[i] <= 1.7:
    X_test.내점당구매건수[i] = 1
  elif X_test.내점당구매건수[i] <= 2.3:
    X_test.내점당구매건수[i] = 2
  elif X_test.내점당구매건수[i] <= 3.4:
    X_test.내점당구매건수[i] = 3  
  else :
    X_test.내점당구매건수[i] = 4

for i in range(len(X_test)):
  if X_test.구매주기[i] <= 4:
    X_test.구매주기[i] = 1
  elif X_test.구매주기[i] <= 13:
    X_test.구매주기[i] = 2
  elif X_test.구매주기[i] <= 28:
    X_test.구매주기[i] = 3  
  else :
    X_test.구매주기[i] = 4
    
# 스케일링
from sklearn.preprocessing import MinMaxScaler    
features = ['총구매액']
scaler = MinMaxScaler()
for feature in features:
    X_test[feature] = scaler.fit_transform(X_test[[feature]])
    
y_pred_proba = rf_model.predict_proba(X_test)[:,1]
# y_pred_proba = best_gr_clf.predict_proba(X_test)[:,1]
y_test = pd.DataFrame(y_pred_proba,columns=['gender'],index=X_df.cust_id)
y_test.reset_index(inplace=True)
print(y_test)
y_test.to_csv('./수험번호.csv', index=False)

      cust_id    gender
0        3500  0.448649
1        3501  0.220024
2        3502  0.171876
3        3503  0.433162
4        3504  0.430526
...       ...       ...
2477     5977  0.460366
2478     5978  0.535143
2479     5979  0.467736
2480     5980  0.352616
2481     5981  0.522482

[2482 rows x 2 columns]


In [10]:
len(y_test[y_test.gender>0.5])

599

In [11]:
599/2482

0.24133763094278807